<a href="https://colab.research.google.com/github/demonlexe/cs4375-group17/blob/main/CS4375_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get your kaggle.json from going to settings in your Kaggle account and down to API and generate token

In [1]:
!pip install kaggle

Upload your kaggle.json file to your Google Drive

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
!kaggle datasets download jacksoncrow/extended-wikipedia-multimodal-dataset

 99% 920M/933M [00:10<00:00, 177MB/s]
100% 933M/933M [00:10<00:00, 91.8MB/s]


Dataset is saved to /content/data

In [20]:
! unzip /content/extended-wikipedia-multimodal-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/Margaret_(singer)/img/meta.json  
  inflating: data/Margaret_(singer)/text.json  
  inflating: data/Margaret_Bondfield/img/meta.json  
  inflating: data/Margaret_Bondfield/text.json  
  inflating: data/Margaret_Fuller/img/meta.json  
  inflating: data/Margaret_Fuller/text.json  
  inflating: data/Margaret_Lea_Houston/img/meta.json  
  inflating: data/Margaret_Lea_Houston/text.json  
  inflating: data/Margaret_Murray/img/meta.json  
  inflating: data/Margaret_Murray/text.json  
  inflating: data/Margarita_with_a_Straw/img/meta.json  
  inflating: data/Margarita_with_a_Straw/text.json  
  inflating: data/Margate_F.C/img/meta.json  
  inflating: data/Margate_F.C/text.json  
  inflating: data/Maria_Rundell/img/meta.json  
  inflating: data/Maria_Rundell/text.json  
  inflating: data/Maria__or,_The_Wrongs_of_Woman/img/meta.json  
  inflating: data/Maria__or,_The_Wrongs_of_Woman/text.json  
  inflating: data/Mariah_Carey/im

Now, let's verify that the data has been added to `content/data`.

In [27]:
import os # accessing directory structure
print(os.getcwd())
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Streaming output truncated to the last 5000 lines.
./data/SMS_Lothringen/text.json
./data/SMS_Lothringen/img/meta.json
./data/Edmund_Sharpe/text.json
./data/Edmund_Sharpe/img/meta.json
./data/Octopus/text.json
./data/Octopus/img/meta.json
./data/George_Harrison/text.json
./data/George_Harrison/img/meta.json
./data/Luton_Town_F.C/text.json
./data/Luton_Town_F.C/img/meta.json
./data/Augusta,_Lady_Gregory/text.json
./data/Augusta,_Lady_Gregory/img/meta.json
./data/Space_Seed/text.json
./data/Space_Seed/img/meta.json
./data/Washington_State_Route_522/text.json
./data/Washington_State_Route_522/img/meta.json
./data/Bill_Russell/text.json
./data/Bill_Russell/img/meta.json
./data/Vesna_Vulovi─З/text.json
./data/Vesna_Vulovi─З/img/meta.json
./data/├Йmile_Lemoine/text.json
./data/├Йmile_Lemoine/img/meta.json
./data/History_of_Norwich_City_F.C/text.json
./data/History_of_Norwich_City_F.C/img/meta.json
./data/Shunzhi_Emperor/text.json
./data/Shunzhi_Emperor/img/meta.json
./data/The_Well_of_Loneli